# 1. Imports & Environment Setup

- **Core Imports:**
  - `os`, `warnings`, `datetime`, `uuid`, `string`, `random`, `hashlib`, `traceback`.

- **LangChain/LangGraph Imports:**
  - `ChatOpenAI`, `StreamingStdOutCallbackHandler`, `TavilySearch`, `TavilySearchResults`, agent/graph/integration tools, message types.

- **Notebook & Display:**
  - `clear_output`, `Markdown` from `IPython.display`.

- **Environment:**
  - Suppresses TensorFlow warnings.
  - Loads environment variables (e.g., API keys) using `python-dotenv`.


In [1]:
import os
import warnings
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_tavily import TavilySearch
from langchain_community.tools import TavilySearchResults
from langchain.agents import create_tool_calling_agent, AgentExecutor
from typing import TypedDict, Dict, List
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from datetime import datetime
from IPython.display import clear_output, Markdown
import traceback
import uuid
import string
import random
import hashlib

In [2]:

# Suppress TensorFlow warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()

def get_api_key(key_name="OPENROUTER_API_KEY"):
    """
    Get API key from environment variables
    """
    api_key = os.getenv(key_name)
    if not api_key:
        raise ValueError(f"Invalid API key: {key_name} not found in environment variables")
    return api_key

def initialize_llm(model_name="meta-llama/llama-3.1-8b-instruct",
                  temperature=0.4,
                  use_streaming=True):
    """
    Initialize LLM
    """
    api_key = get_api_key()
    callbacks = [StreamingStdOutCallbackHandler()]
    llm = ChatOpenAI(
        model_name=model_name,
        temperature=temperature,
        streaming=use_streaming,
        callbacks=callbacks,
        openai_api_key=api_key,
        openai_api_base="https://openrouter.ai/api/v1"
    )
    return llm

llm = initialize_llm()

python-dotenv could not parse statement starting at line 12


In [3]:
def save_file(data, filename, uniqueness_method="uuid"):

    folder_name = "Travel Plans"  # Folder to store output files
    os.makedirs(folder_name, exist_ok=True)  # Creates the folder if it doesn't exist
    
    # Generate unique identifier based on selected method
    if uniqueness_method == "uuid":
        # Generate a UUID (universally unique identifier)
        unique_id = str(uuid.uuid4())[:8]
        
    elif uniqueness_method == "random_string":
        # Generate a random string of letters and digits
        chars = string.ascii_letters + string.digits
        unique_id = ''.join(random.choice(chars) for _ in range(8))
        
    elif uniqueness_method == "hash":
        # Create a hash based on content and current time
        content_hash = hashlib.md5((data + str(datetime.now())).encode()).hexdigest()[:8]
        unique_id = content_hash
        
    elif uniqueness_method == "counter":
        # Use an incrementing counter for files with the same base name
        counter = 1
        while True:
            test_filename = f"{filename}_{counter}.md"
            test_path = os.path.join(folder_name, test_filename)
            if not os.path.exists(test_path):
                unique_id = str(counter)
                break
            counter += 1
            
    elif uniqueness_method == "datetime":
        # Original datetime method
        unique_id = datetime.now().strftime("%Y%m%d%H%M%S")
        
    else:
        # Default to UUID if an invalid method is specified
        unique_id = str(uuid.uuid4())[:8]
    
    # Create the final filename with the unique identifier
    final_filename = f"{filename}_{unique_id}.md"
    file_path = os.path.join(folder_name, final_filename)
    
    # Save the data to the file in the specified path
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(data)
        print(f"File '{file_path}' created successfully.")
    
    # Return the full path of the saved file
    return file_path

def show_md_file(file_path):
    with open(file_path, "r", encoding = "utf-8") as file:
        content = file.read()

    display(Markdown(content))

In [4]:
class PlannerState(TypedDict):
    travel_preferences: Dict
    destination_options: List[Dict]
    selected_destination: Dict
    budget_plan: Dict
    itinerary: List[Dict]
    bookings: Dict
    feedback: Dict
    final_trip: Dict
    messages: List
    status: str

In [5]:
def input_collector(state: PlannerState) -> PlannerState:
    """Collects and organizes user's travel preferences."""
    messages = state["messages"]
    
    # Extract the latest human message
    human_message = next((m for m in reversed(messages) if isinstance(m, HumanMessage)), None)
    system_prompt = """You are a travel input collector. Extract travel preferences from the user's message.
    Include: destination interests, travel dates, budget range, number of travelers, 
    accommodation preferences, activity interests, and any special requirements.
    """
    
    response = llm.invoke([
        SystemMessage(content = system_prompt),
        human_message
    ])
    
    # Parse the response into structured travel preferences
    preferences = {
        "raw_input": human_message.content,
        "processed_preferences": response.content,
        "timestamp": datetime.now().isoformat()
    }
    
    # Update state
    state["travel_preferences"] = preferences
    state["messages"].append(AIMessage(content=f"I've collected your travel preferences. Moving on to destination research."))
    state["status"] = "preferences_collected"
    
    return state

In [6]:
def destination_research(state: PlannerState) -> PlannerState:
    """Researches and suggests destinations based on user preferences"""
    preferences = state["travel_preferences"]
    system_prompt = """
    You are a destination research expert. Based on the user's travel preferences, suggest 
    3-5 suitable destinations. For each destination, provide:
    - Name and brief description
    - Why it matches their preferences
    - Best time to visit
    - Estimated overall cost level
    - Key attractions
    """
    
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content = f"Travel Preferences: {preferences}")
    ])
    
    # Process the response into structured destination options
    destination_options = {
        "options": response.content,
        "research_timestamp": datetime.now().isoformat()
    }
    
    # Update State
    state["destination_options"] = destination_options
    state["messages"].append(AIMessage(content=f"I've researched some destinations that match your preferences. Moving on to budget planning."))
    state["status"] = "awaiting_destination_selection"
    
    return state